# EARTH ENGINE PYTHON API TO POSTGRESQL

### Definir a data de interesse:

In [1]:
dtInicio = '2016-01-01';
dtFim = '2017-01-01';
data = ee.DateRange(ee.Date(dtInicio),ee.Date(dtFim).advance(1,'month'))

### Inicializar a conexão com a a plataforma do Earth Engine

In [3]:
import ee
print("Iniciando conexão com EE...");
ee.Initialize();
print("Conectado!");

Iniciando conexão com EE...
Conectado!


#### Região de estudo

In [ ]:
regiao = ee.Feature(ee.FeatureCollection("users/odraitaipu/bracos/ocoi").union().first()).set('nome','Ocoí')

#### Coleção
Sentinel 2, filtrando apenas na região de interesse e na data selecionada, com até 5% de nuvem

In [4]:
collection = ee.ImageCollection('COPERNICUS/S2').filterBounds(regiao.geometry()).filterDate(data).filter(ee.Filter.lessThanOrEquals('CLOUDY_PIXEL_PERCENTAGE',5)).select(['B8','B4','B11']).sort('system:time_start');

AQUI ONDE A MAGICA ACONTECE

In [8]:
def getMacrofitas(image):

	image = ee.Image(image).divide(10000);

	bands = image.bandNames();
	ndvi = image.normalizedDifference();

	soma = ndvi.reproject(image.select('B4').projection()).reduceResolution(ee.Reducer.sum());
	porcentagem = soma.divide(ndvi).multiply(100).round();
	clippedNdvi = ndvi.updateMask(ndvi.where(porcentagem.lte(99),0));

	output = ee.Image(0).where(clippedNdvi.gte(0.2),1);
	result = output.updateMask(output).rename(['macrofitas']).reproject(image.select('B4').projection());

	return result;

Mapeia a coleção das imagens brutas de satélite em uma nova coleção de imagens de 'macrófitas'

In [9]:
collectMacrofita = ee.ImageCollection(collection.map(getMacrofitas));

### IMG MACRÓFITA 1 + IMG MACRÓFITA 2 + IMG MACRÓFITA 3... = IMG RESULTADO 

| 0 1 1 |&nbsp;&nbsp;|0 0 1 |&nbsp;&nbsp;|0 0 1 |&nbsp;&nbsp;&nbsp;&nbsp; 0 1 3<br> 
| 0 0 1 |+|0 0 1 |+|0 0 0 | = &nbsp;0 0 2<br>
| 0 0 0 |&nbsp;&nbsp;|0 0 1 |&nbsp;&nbsp;|0 0 0 |&nbsp;&nbsp;&nbsp;&nbsp; 0 0 1<br>


In [ ]:
imgMacrofitas = collectMacrofita.sum();